In [1]:
from awrams.utils.processing.extract import extract
from awrams.utils.gis import ShapefileDB,extent_from_record
from awrams.utils.extents import get_default_extent
import os, sys
import pandas as pd
import pickle
from itertools import product
from datetime import datetime, timedelta

In [2]:
# parameter
var_name = 'rain_day'

In [3]:
## output folder path
customer_dir = '/g/data/er4/zk6340/Hydro_projection/'

In [16]:
### Set the customer folder and shapefile name
shp_name = 'New England Combined'
#shp_name = 'Burdekin'
#shp_name = 'Cairns Catchment'
# shp_name = 'Central West River Catchment'

In [17]:
## Data Request dir folder path
#DataRequest_dir = '/g/data/er4/zk6340/Hydro_projection/%s/'%(shp_name)
DataRequest_dir = '/g/data/er4/zk6340/Hydro_projection/%s/'%(shp_name)

In [18]:
## Read shape file
shp = DataRequest_dir + shp_name + '.shp'
sdb = ShapefileDB(shp)
sdf = sdb.get_records_df()
sdf

,AlbersArea,AttrRel,AttrSource,Division,FSource,FeatRel,HydroID,OBJECTID,PlanAcc,RivRegName,RivRegNum,Shape_Area,Shape_Leng,SourceID,SrcFCName,SrcFType,SrcType,Symbol,TextNote,name
0,3.937612e+10,2019/08/21,Bureau of Meteorology,Murray-Darling Basin,Bureau of Meteorology,2019/08/21,43637150,238,15,Combined,20,3.712643,16.507778,0,None,None,0,0,None,New England Combined


In [19]:
## Read shape file
shp = DataRequest_dir + shp_name + '.shp'
sdb = ShapefileDB(shp)
sdf = sdb.get_records_df()
sdf

,AlbersArea,AttrRel,AttrSource,Division,FSource,FeatRel,HydroID,OBJECTID,PlanAcc,RivRegName,RivRegNum,Shape_Area,Shape_Leng,SourceID,SrcFCName,SrcFType,SrcType,Symbol,TextNote,name
0,3.937612e+10,2019/08/21,Bureau of Meteorology,Murray-Darling Basin,Bureau of Meteorology,2019/08/21,43637150,238,15,Combined,20,3.712643,16.507778,0,None,None,0,0,None,New England Combined


### Create the extents from above the shape file - only needed if it does not exist

In [20]:
#Create the extents and save as pkl file if pkl is not available
edef = get_default_extent()
emap = {}
for i,e in enumerate(sdf['name']): 
    print('Dealing with {}'.format(i))
    try:
        emap[e] = extent_from_record(sdb.records[i],edef)
#     except Exception as err:
#         print('Error in delineation')
pickle.dump(emap, open(DataRequest_dir + shp_name + '.pkl', 'wb'))

Dealing with 0


AttributeError: 'NoneType' object has no attribute 'Transform'

In [13]:
sdb.records[i]

<osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x7f8ed5543690> >

In [ ]:
edef

In [ ]:
edef

In [ ]:
sdb.records[i]

### Source the existing extents file for the above shape file

In [ ]:
emap = pickle.load(open(DataRequest_dir + shp_name + '.pkl', 'rb'))

In [ ]:
emap

In [ ]:
emap

### period to extract

In [ ]:
period = {'hist':['1985-01-01 12:00:00','2005-12-31 12:00:00']}
#period = {'scn1':['2031-01-01 12:00:00','2050-12-31 12:00:00'],'scn2':['2051-01-01 12:00:00','2070-12-31 12:00:00']}

### extract the data

In [ ]:
# Participating GCMs, Bias correction approaches, Emission scenarios
gcms = ['CNRM-CERFACS-CNRM-CM5','CSIRO-BOM-ACCESS1-0','MIROC-MIROC5','NOAA-GFDL-GFDL-ESM2M']
bias_corr = ['CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP','r240x120-ISIMIP2b-AWAP', 'r240x120-MRNBC-AWAP', 'r240x120-QME-AWAP']
emission = ['historical']
#emission = ['rcp45','rcp85']

In [ ]:
# Generate all the emsemble identities
values = [(i, k, x) for i, k, x in product(gcms, emission, bias_corr)]

In [ ]:
#Function to get to the path 
def get_path(parameter,which_gcm,which_emission,which_bias_corr):
    if parameter == 'qtot':
        path = os.path.join('/g/data/wj02/COMPLIANT/HMOUTPUT/output/AUS-5/BoM/AWRALv6-1-%s/%s/r1i1p1/%s/latest/day/%s/'%(which_gcm,which_emission,which_bias_corr,parameter))
    else:
        path = os.path.join('/g/data/wj02/COMPLIANT/HMINPUT/output/AUS-5/BoM/%s/%s/r1i1p1/%s/latest/day/%s/'%(which_gcm,which_emission,which_bias_corr,parameter))
    return(path)

In [ ]:
# historical period
period_hist = pd.date_range('1 jan 1976', '31 dec 2005', freq='D')

In [ ]:
# Path to historical data
if parameter == 'qtot':
    data_hist = '/g/data/er4/exv563/hydro_projections/data/drought_2018_analysis/awra_v6_static_winds/daily'
else:
    data_hist = '/g/data/er4/data/CLIMATE/rain_day'

In [ ]:
ds_produced = []
for val in enumerate(values): 
    which_catchment = val[1][0]
    print(which_catchment)
#         period_projected = pd.date_range(future_period[val[1][1]][0], future_period[val[1][1]][1], freq='D')
    catchment = {}
    for key in emap:
        if which_catchment in key:
            catchment[key] = emap[key]
    #Annual max for historical data                
    pattern = data_hist + '/%s*.nc' % var_name
    ds_hist = extract(data_hist, pattern, var_name, catchment, period_hist)
    data_mean_hist = ds_hist[which_catchment].values.mean()
    data_max_hist = ds_hist[which_catchment].values.max()
    data_annual_max_hist = ds_hist.resample(rule='A-DEC').max()

In [ ]:
# Get the calculation done!
ensb_together = []
for ensb in enumerate(ensb_mem):
    ds_produced = []
    for val in enumerate(values): 
        which_catchment = val[1][0]
        print(which_catchment)
#         period_projected = pd.date_range(future_period[val[1][1]][0], future_period[val[1][1]][1], freq='D')
        catchment = {}
        for key in emap:
            if which_catchment in key:
                catchment[key] = emap[key]
        #Annual max for historical data                
        pattern = data_hist + '/%s*.nc' % var_name
        ds_hist = extract(data_hist, pattern, var_name, catchment, period_hist)
        data_mean_hist = ds_hist[which_catchment].values.mean()
        data_max_hist = ds_hist[which_catchment].values.max()
        data_annual_max_hist = ds_hist.resample(rule='A-DEC').max()
        
#         #Annual max for future data    
#         input_path_gcm = get_path(variable,ensb[1][0],ensb[1][1],ensb[1][2])
#         input_file = get_file(ensb[1][0],ensb[1][2],ensb[1][1],variable) 
#         pattern_gcm = input_path_gcm + '/' + input_file
#         ds_gcm = extract(input_path_gcm, pattern_gcm, variable, catchment, period_projected)
#         if var_name == 'rain_day':
#             ds_convrt = ds_gcm*86400
#         else:
#             ds_convrt = ds_gcm
#         data_mean_future = ds_convrt[which_catchment].values.mean()
#         data_max_future = ds_convrt[which_catchment].values.max()
#         data_annual_max_future = ds_convrt.resample(rule='A-DEC').max()
        break
    break

In [ ]:
for val in enumerate(values):    
        for key, value in period.items():
                    print(val[1])
                    v = parameter
                    input_path = get_path(parameter,val[1][0],val[1][1],val[1][2])
                    pattern = input_path + '*%s*.nc' % v                    
                    extraction_period = pd.date_range(value[0], value[1], freq='D')
                    print('input file path:', pattern)   
                    ddf = extract(input_path, pattern, v, emap, extraction_period)
                    # save daily/monthly/annual data in mm
                    print('writing data for', v)
                    dir_name = customer_dir+'%s_VIC_%s_%s'%(v,val[1][1],extraction_period[-1].strftime('%Y%m%d'))
                    os.makedirs(dir_name, exist_ok=True)
                    filename_daily = '%s_%s_%s_%s_daily.csv'%(v,shp_name,val[1][2],val[1][0])
                    filename_monthly = '%s_%s_%s_%s_monthly.csv'%(v,shp_name,val[1][2],val[1][0])
                    filename_annual = '%s_%s_%s_%s_annual.csv'%(v,shp_name,val[1][2],val[1][0])
                    ddf.to_csv((os.path.join(dir_name,filename_daily)), index=True)  
                    ddf.resample('MS').sum().to_csv((os.path.join(dir_name,filename_monthly)), index=True) 
                    ddf.resample(rule='A-DEC').sum().to_csv((os.path.join(dir_name,filename_annual)), index=True) 
                    break
        break

In [ ]:
if (ddf.isnull().all() == False):
    print ('yes')